In [1]:
import numpy as np
import pandas as pd

In [2]:
name = "Reddit Bot"

#Enter Reddit username and password. 
username = "a__man"
password = "SEcL9Qo5iVOo"

#After registering script enter the client id and client secret provided.
cid = "jJ8t4gCXNMETpg"
csec = "QM7u6lG2vJjOQrgJmEQrtifbNUM"

In [3]:
#Defining functions.
import praw
import re
import datetime
import matplotlib.pyplot as plt

i = [name, username, password, cid, csec]

#Extract original and current scores of the linked comment, along with timestamps, extra info and plots.
def link_comparison(subreddit_name, n_posts=25, quotes_score=False, i=i):
    r = praw.Reddit(user_agent=i[0], client_id=i[3], client_secret=i[4], username=i[1], password=i[2])
    subreddit = r.subreddit(subreddit_name)

    #Return relevant information from a comment.
    def comment_data(comment):
        final_score = comment.score
        if comment.author:
            deleted = 0
        else:
            deleted = 1
        time = datetime.datetime.utcfromtimestamp(comment.created_utc)
        return [time, final_score, deleted]
    
    def parse_score(title):
        title = title.replace(' ', '')
        init_score = re.findall(r'\+\d+|\d+\+|\[\d+\]', title)
        init_score = [int(re.sub('[^0-9]','', i)) for i in init_score]
        
        if len(init_score) == 0:
            return [None]
        else:
            return init_score
        
    data = []
    for submission in subreddit.hot(limit=n_posts):
        if submission.selftext:
            continue
        
        post_id = submission.id
        link_score = submission.score
        time = datetime.datetime.utcfromtimestamp(submission.created)
        
        url = submission.url
        try:
            comment = r.comment(url=url)
        except praw.exceptions.ClientException:
            continue

        if quotes_score:
            quoted_score = parse_score(submission.title)
            scores = len(quoted_score)
            if scores > 1:
                comment.refresh()
                score = comment.score
                children = comment.replies
                if children:
                    for i in range(scores - 1):
                        data.append([post_id, link_score, quoted_score[i + 1], time] + comment_data(children[i]))    
        else:
            quoted_score = [None]
        
        data.append([post_id, link_score, quoted_score[0], time] + comment_data(comment))
        
    df = pd.DataFrame(data)
    df.columns = ['id', 'link_score', 'quoted_score', 'link_time', 'comment_time', 'final_score', 'deleted']
    
    return df

In [4]:
link_comparison('bestof')

,id,link_score,quoted_score,link_time,comment_time,final_score,deleted
0,asbnsi,4833,None,2019-02-19 15:37:31,2019-02-19 11:51:25,6060,0
1,as9pk9,2862,None,2019-02-19 12:07:10,2019-02-19 01:31:27,4350,0
2,asdnoi,592,None,2019-02-19 18:33:05,2019-02-19 14:24:09,65,1
3,asjmht,67,None,2019-02-20 03:58:03,2019-02-20 01:01:23,1,0
4,asdmh5,296,None,2019-02-19 18:30:08,2019-02-19 05:34:48,200,0
5,as2xwh,4163,None,2019-02-18 22:26:16,2019-02-18 21:25:39,3823,0
6,asfweq,40,None,2019-02-19 21:49:55,2019-02-19 17:52:29,1088,0
7,ascmed,51,None,2019-02-19 17:05:16,2019-01-01 16:23:29,91,0
8,asc82p,48,None,2019-02-19 16:29:39,2019-02-19 07:41:08,3,0
9,asa3si,51,None,2019-02-19 12:55:02,2019-02-17 17:39:10,201,0
